In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import re
from nltk.corpus import stopwords
import pickle

In [17]:
IT_Ticket_cleaned = pd.read_excel('Cleaned_data.xlsx')
IT_Ticket_cleaned.shape

(8432, 12)

In [27]:
# IT_Ticket_cleaned = IT_Ticket_cleaned[IT_Ticket_cleaned['Assignment RE-group'] != 'Others']
Empty_Rows = IT_Ticket_cleaned[IT_Ticket_cleaned.isnull().any(axis=1)].index
IT_Ticket_cleaned.drop(Empty_Rows,axis = 0,inplace = True)

In [28]:
IT_Ticket_cleaned['Assignment group'].unique()

array(['GRP_0', 'GRP_1', 'GRP_3', 'GRP_4', 'GRP_5', 'GRP_6', 'GRP_7',
       'GRP_8', 'GRP_9', 'GRP_10', 'GRP_11', 'GRP_12', 'GRP_13', 'GRP_14',
       'GRP_15', 'GRP_16', 'GRP_17', 'GRP_18', 'GRP_19', 'GRP_2',
       'GRP_20', 'GRP_21', 'GRP_22', 'GRP_23', 'GRP_24', 'GRP_25',
       'GRP_26', 'GRP_27', 'GRP_28', 'GRP_29', 'GRP_30', 'GRP_31',
       'GRP_33', 'GRP_34', 'GRP_35', 'GRP_36', 'GRP_37', 'GRP_38',
       'GRP_39', 'GRP_40', 'GRP_41', 'GRP_42', 'GRP_43', 'GRP_44',
       'GRP_45', 'GRP_46', 'GRP_47', 'GRP_49', 'GRP_50', 'GRP_51',
       'GRP_52', 'GRP_53', 'GRP_54', 'GRP_55', 'GRP_56', 'GRP_57',
       'GRP_58', 'GRP_59', 'GRP_60', 'GRP_61', 'GRP_32', 'GRP_62',
       'GRP_63', 'GRP_64', 'GRP_48', 'GRP_65', 'GRP_66', 'GRP_67',
       'GRP_68', 'GRP_69', 'GRP_70', 'GRP_71', 'GRP_72', 'GRP_73'],
      dtype=object)

In [29]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(IT_Ticket_cleaned['Cleaned'],IT_Ticket_cleaned['Assignment group'],test_size=0.25,random_state=10)

In [30]:
import pickle
for i in range(6):
    pickled = open("elmo_full_train_" + str(i+1) +".pickle","rb")
    temp = pickle.load(pickled)
    if i == 0:
        elmo_train_vectors = temp
    else:
        elmo_train_vectors = np.concatenate((elmo_train_vectors,temp), axis = 0)

In [31]:
for i in range(6):
    pickled = open("elmo_full_test_" + str(i+1) +".pickle","rb")
    temp = pickle.load(pickled)
    if i == 0:
        elmo_test_vectors = temp
    else:
        elmo_test_vectors = np.concatenate((elmo_test_vectors,temp), axis = 0)

In [32]:
X = np.concatenate((elmo_train_vectors,elmo_test_vectors), axis = 0)
Y = np.concatenate((y_train,y_test),axis = 0)

In [33]:
from sklearn.preprocessing import LabelEncoder
label_enc = LabelEncoder()
Y = label_enc.fit_transform(Y)

In [34]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.25,random_state=10)

In [36]:
from tensorflow.keras.utils import to_categorical
y_test= to_categorical(y_test,num_classes = 74)
y_train= to_categorical(y_train,num_classes = 74)

In [44]:
from tensorflow.keras.layers import LSTM,Embedding,Flatten,Input,Dense,InputLayer
from tensorflow.keras.models import Sequential

model = Sequential()
model.add(InputLayer(input_shape = (1024,)))
model.add(Dense(500))
model.add(Dense(74,activation='softmax'))
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 500)               512500    
_________________________________________________________________
dense_7 (Dense)              (None, 74)                37074     
Total params: 549,574
Trainable params: 549,574
Non-trainable params: 0
_________________________________________________________________
None


In [45]:
from tensorflow.keras.optimizers import Adam
adam = Adam()

model.compile(optimizer=adam,loss = "categorical_crossentropy",metrics=['accuracy'])

In [46]:
model.fit(x_train,y_train,batch_size=64,epochs=20,validation_data=(x_test,y_test))

Epoch 1/20
99/99 [==============================] - 1s 6ms/step - loss: 3.5810 - accuracy: 0.4455 - val_loss: 2.7016 - val_accuracy: 0.5419
Epoch 2/20
99/99 [==============================] - 0s 5ms/step - loss: 2.4814 - accuracy: 0.5548 - val_loss: 2.2475 - val_accuracy: 0.5371
Epoch 3/20
99/99 [==============================] - 0s 5ms/step - loss: 1.8899 - accuracy: 0.5884 - val_loss: 2.6560 - val_accuracy: 0.5490
Epoch 4/20
99/99 [==============================] - 0s 5ms/step - loss: 1.8283 - accuracy: 0.6010 - val_loss: 2.0759 - val_accuracy: 0.5710
Epoch 5/20
99/99 [==============================] - 0s 5ms/step - loss: 1.7546 - accuracy: 0.6168 - val_loss: 1.9495 - val_accuracy: 0.5624
Epoch 6/20
99/99 [==============================] - 0s 5ms/step - loss: 1.6340 - accuracy: 0.6349 - val_loss: 1.8212 - val_accuracy: 0.5695
Epoch 7/20
99/99 [==============================] - 0s 5ms/step - loss: 1.4647 - accuracy: 0.6526 - val_loss: 1.7835 - val_accuracy: 0.5976
Epoch 8/20
99/99 [==

In [ ]:
import fasttext

model = fasttext.train_supervised(input="traindata.txt") 